In [1]:
import pandas as pd
from pandasql import sqldf
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot

/Users/polaris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv("/Users/polaris/NotebookProjects/01_sales_predict/query_result.csv")
columns=[re.sub("pmt_sku_ss.", "", col) for col in df.columns]
columns
df.columns = columns

In [8]:
df = df.fillna(0)

In [9]:
df.head()

,inv_date,brand_no,product_code,product_no,product_year_name,product_season_name,style_name,style_category_name,color_name,main_mtrl_name,...,amount,tag_amount,in_inv,prod_xssl,main_qty,main_amount,main_tag_amount,main_prod_xssl,store_num,main_store_num
0,20170101,ST,S59G6101DU1AM7,20161205000154,2017,春,假日休闲,~,黑色,牛皮革,...,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,14,14
1,20170101,ST,SZP9A703DU1AQ7,6121637,2017,春,休闲上班,~,黑色,皮,...,0.0,0.0,68.0,0.0,0.0,0.0,0.0,0.0,11,11
2,20170101,ST,SZP9A704DL1AK7,6121642,2017,春,休闲上班,~,红色,皮,...,0.0,0.0,87.0,0.0,0.0,0.0,0.0,0.0,14,14
3,20170101,ST,SZP9A704DU1AK7,6121641,2017,春,休闲上班,~,黑色,皮,...,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,6,6
4,20170101,ST,SZP9AG59DM8AQ7,6121302,2017,春,优雅商务,~,深蓝,皮,...,0.0,0.0,85.0,0.0,0.0,0.0,0.0,0.0,18,18


In [23]:
q="""select in_inv,store_num,main_store_num+1 as main_store_num, main_prod_xssl from df where product_no = 6121637 and main_prod_xssl>0"""
df_sub = sqldf(q)

In [24]:
df_sub.head(10)

,in_inv,store_num,main_store_num,main_prod_xssl
0,92.0,18,19,0.062500
1,750.0,148,149,0.008772
2,875.0,171,172,0.005917
3,970.0,193,194,0.013252
4,1064.0,213,214,0.011457
5,1069.0,213,214,0.009756
6,1067.0,213,214,0.004902
7,1064.0,213,214,0.016238
8,1060.0,213,214,0.010236
9,1058.0,213,214,0.009709


In [25]:

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df_sub.values)
pd.DataFrame(scaled).head(10)

,0,1,2,3
0,0.049932,0.026936,0.030201,0.052093
1,0.500000,0.464646,0.466443,0.004696
2,0.585499,0.542088,0.543624,0.002178
3,0.650479,0.616162,0.617450,0.008648
4,0.714774,0.683502,0.684564,0.007065
5,0.718194,0.683502,0.684564,0.005565
6,0.716826,0.683502,0.684564,0.001282
7,0.714774,0.683502,0.684564,0.011283
8,0.712038,0.683502,0.684564,0.005988
9,0.710670,0.683502,0.684564,0.005523


In [32]:
x, y  = scaled[:-1,:-1],scaled[:-1,-1]

In [36]:
y.shape

(237,)

In [39]:
from numpy import concatenate
y = y.reshape((y.shape[0],1))
inv_yhat = concatenate((x,y), axis=1)

pd.DataFrame(inv_yhat).head(10)

,0,1,2,3
0,0.049932,0.026936,0.030201,0.052093
1,0.500000,0.464646,0.466443,0.004696
2,0.585499,0.542088,0.543624,0.002178
3,0.650479,0.616162,0.617450,0.008648
4,0.714774,0.683502,0.684564,0.007065
5,0.718194,0.683502,0.684564,0.005565
6,0.716826,0.683502,0.684564,0.001282
7,0.714774,0.683502,0.684564,0.011283
8,0.712038,0.683502,0.684564,0.005988
9,0.710670,0.683502,0.684564,0.005523


In [40]:
reform = scaler.inverse_transform(inv_yhat)

In [41]:
pd.DataFrame(reform).head(10)

,0,1,2,3
0,92.0,18.0,19.0,0.062500
1,750.0,148.0,149.0,0.008772
2,875.0,171.0,172.0,0.005917
3,970.0,193.0,194.0,0.013252
4,1064.0,213.0,214.0,0.011457
5,1069.0,213.0,214.0,0.009756
6,1067.0,213.0,214.0,0.004902
7,1064.0,213.0,214.0,0.016238
8,1060.0,213.0,214.0,0.010236
9,1058.0,213.0,214.0,0.009709
